In [2]:
from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
from tqdm import tqdm

### 구름edu-강의명, 가격, 강사, 평점, url

In [136]:
title_list = [] #강의명
price_list = [] #가격
teacher_list = [] #강사
rating_list= [] #평점
url_list = [] #url

for i in tqdm(range(1,21)):
    #사이트 페이지별
    url1 = "https://edu.goorm.io/category/programming?page="+str(i)+"&sort=newest"
    driver = wb.Chrome()
    driver.get(url1)
    soup = bs(driver.page_source,"lxml")
    time.sleep(1)
    #각 요소 추출
    price = soup.select("div.RfUd-z._1AV9t8.card-footer > div > span._3vh60A")
    title = soup.select("div > div.obzTi8 > div.kV2LiJ")
    teacher = soup.select('._3SzRPA > .xaJHLa')
    rating = soup.select('._2KWt9f._3SwFuE')
    url = soup.select("._1xnzzp")
    time.sleep(1)
    
    #각 요소 리스트에 저장
    for i in range(len(title)):
        title_list.append(title[i].text)
        price_list.append(price[i].text)
        teacher_list.append(teacher[i].text)
        rating_list.append(rating[i].text)
        url_list.append('https://edu.goorm.io'+url[i]['href'])
    time.sleep(1)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:10<00:00, 10.81s/it]


### 본진 -> 수정 필요

In [67]:
url1 = "https://edu.goorm.io/category/programming?page=2&sort=newest"
driver = wb.Chrome()
driver.get(url1)
time.sleep(1)
#각 항목마다 들어가기
for i in range(1,21):
    try:
        btn = driver.find_element_by_css_selector('div.Aj2j_L > div:nth-child('+str(i)+')>a')
        btn.click()
        time.sleep(3)
        soup = bs(driver.page_source,"lxml")
        #카테고리(cate), 리뷰 초기화
        cate = soup.select('div:nth-child(2) > div._2yM5um')
        review_list = []
        #카테고리 저장
        for j in range(len(cate)):
            cate_list.append(cate[j].text)
        #수강후기(review)
        review = soup.select('#app > section > div._1h8WRN._2fKbgw._1f1oP_ > div._3IohZ2 > div > div > div > ul > li')
        #리뷰가 없을 경우를 생각(오류있는 듯, 처리 안됨)
        if review == None:
            review_list.append("리뷰없음")
        else:
        #각 리스트의 텍스트만 다른 리스트에 저장
            for i in range(len(review)):
                review_list.append(review[i].text)
                #저장된 리뷰를 하나로 모아서 저장
        goorum_review_list.append(review_list)
        driver.back()
        time.sleep(2)
    except:
        body = driver.find_element_by_css_selector('body') 
        body.send_keys(Keys.PAGE_DOWN) # 페이지 다운
        time.sleep(1) # 1초 딜레이
        btn = driver.find_element_by_css_selector('div.Aj2j_L > div:nth-child('+str(i)+')>a')
        btn.click()
        time.sleep(3)
        #카테고리(cate), 리뷰 초기화
        cate = soup.select('div:nth-child(2) > div._2yM5um')
        review_list = []
        #카테고리 저장
        for j in range(len(cate)):
            cate_list.append(cate[j].text)
        try:
            #수강후기(review)
            review = soup.select('p._1KbNtk.card-text')
            #각 리스트의 텍스트만 다른 리스트에 저장
            for i in range(len(review)):
                review_list.append(review[i].text)
            #저장된 리뷰를 하나로 모아서 저장
            goorum_review_list.append(review_list)
        except:
            review_list.append("리뷰없음")
        driver.back()
        time.sleep(2)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.Aj2j_L > div:nth-child(3)>a"}
  (Session info: chrome=94.0.4606.81)


### 리뷰 제외 후 카테고리만 페이지별로

#### 몇개 항목이 짜증나게 뒤로가기를 두번해야 함. 2페이지의 3,4,5번인듯....

In [138]:
cate_list2 = []
for i in tqdm(range(1,21)):
    for k in tqdm(range(1,21)):
    url1 = "https://edu.goorm.io/category/programming?page="+str(i)+"&sort=newest"
    driver = wb.Chrome()
    driver.get(url1)
    soup = bs(driver.page_source,"lxml")
    time.sleep(1)
    btn = driver.find_element_by_css_selector('div.Aj2j_L > div:nth-child('+str(k)+')>a')
    btn.click()
    time.sleep(3)
    soup = bs(driver.page_source,"lxml")
    #카테고리(cate)
    cate = soup.select('div:nth-child(2) > div._2yM5um')
    #카테고리 저장
    for j in range(len(cate)):
        cate_list2.append(cate[j].text)
    driver.close()

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [03:47<00:00, 11.36s/it]


In [59]:
dic = {'제목':title_list, '가격':price_list, '강사':teacher_list, 'URL': url_list, '카테고리':cate_list}
df = pd.DataFrame(dic)
df

,제목,가격,강사,URL,카테고리,리뷰
0,HTML5 & CSS3 기초 문법 올인원,무료,개발자 이도해,https://edu.goorm.io/lecture/28488/html5-css3-...,웹 프로그래밍 - 프로그래밍,[목소리도 차분하시고 조곤조곤 설명을 잘 해주셔서 즐겁게 공부하고 있어요 처음이라 ...
1,컴퓨터 GUI 앱을 만들면서 tkinter 배우기,무료,행보칸코딩,https://edu.goorm.io/lecture/28428/컴퓨터-gui-앱을-...,프로그래밍 - 프로그래밍 기초,[wonderful]
2,[하루10분|Web-App] 생애최초 웹프레임워크 Ruby on Rails,무료,판다코딩,https://edu.goorm.io/lecture/28494/하루10분-web-a...,웹 프로그래밍 - 프로그래밍,[짧고 굵은 강의입니다.]
3,실전!웹사이트제작 Step By Step! (Basic 강좌 : 쌩초보 코스),"₩10,000",J.young,https://edu.goorm.io/lecture/28099/실전-웹사이트제작-s...,프로그래밍 - 프로그래밍 기초,[]
4,실전!웹사이트제작 Step By Step! (클론코딩 : 포메인사이트제작 - 풀페이지),"₩28,000",J.young,https://edu.goorm.io/lecture/28100/실전-웹사이트제작-s...,웹 프로그래밍 - 프로그래밍,[]
5,실전!웹사이트제작 Step By Step! (클론코딩 : crew a la Mode...,"₩28,000",J.young,https://edu.goorm.io/lecture/28057/실전-웹사이트제작-s...,웹 프로그래밍 - 프로그래밍,[]
6,퇴근후 텐서플로: Basic,무료,퇴근후딴짓,https://edu.goorm.io/lecture/28058/퇴근후-텐서플로-basic,프로그래밍 - 인공지능,"[Good, 입문에 최적화된 강좌입니다.!\n, 친절한 설명 👍 굳!!]"
7,입문자를 위한 CSS 기초 강의,무료,유노코딩,https://edu.goorm.io/lecture/27976/입문자를-위한-css...,웹 프로그래밍 - 프로그래밍,"[너무 상세하고 친절하게 알려주셔서 좋아여 목소리도 너무좋으시네요 짱짱\n1, 너~..."
8,입문자를 위한 HTML 기초 강의,무료,유노코딩,https://edu.goorm.io/lecture/27971/입문자를-위한-htm...,웹 프로그래밍 - 프로그래밍,"[너무 상세하고 친절하게 알려주셔서 좋아여 목소리도 너무좋으시네요 짱짱\n1, 너~..."
9,Vuetify Admin Template 만들기 - 실전편,무료,짐코딩 GYM CODING,https://edu.goorm.io/lecture/27961/vuetify-adm...,웹 프로그래밍 - 프로그래밍,[]


In [60]:
df.to_csv("구름edu_2페이지.csv", encoding="utf-8-sig")

In [149]:
print(len(title_list2))
print(len(price_list2))
print(len(teacher_list2))
print(len(rating_list2))
print(len(url_list2))
print(len(cate_list2))

20
20
20
20
20
20


In [150]:
dic2 = {'제목':title_list2, '가격':price_list2, '강사':teacher_list2, 'URL': url_list2, '카테고리':cate_list2}
df = pd.DataFrame(dic2)
df

,제목,가격,강사,URL,카테고리
0,실전! 웹사이트 제작 Step by Step - 호텔 라한 Application C...,"₩27,500",J.young,https://edu.goorm.io/lecture/26324/실전-웹사이트-제작-...,웹 프로그래밍 - 프로그래밍
1,딥러닝 네트워크의 연산,"₩39,000",Shin's Lab,https://edu.goorm.io/lecture/26039/딥러닝-네트워크의-연산,프로그래밍 - 인공지능
2,데이터 시각화를 위한 파이썬,"₩39,000",Shin's Lab,https://edu.goorm.io/lecture/24603/데이터-시각화를-위한...,프로그래밍 - 데이터 추출/분석
3,딥러닝을 위한 파이썬 레벨2,"₩39,000",Shin's Lab,https://edu.goorm.io/lecture/26038/딥러닝을-위한-파이썬...,프로그래밍 - 인공지능
4,딥러닝을 위한 파이썬 레벨1,"₩35,100",Shin's Lab,https://edu.goorm.io/lecture/26037/딥러닝을-위한-파이썬...,프로그래밍 - 인공지능
5,모의해킹 취업 준비하는데 알아두면 좋은 것,무료,보안프로젝트,https://edu.goorm.io/lecture/25879/모의해킹-취업-준비하...,프로그래밍 - 보안
6,파이썬 개발자를 위한 go언어 기초,무료,행보칸코딩,https://edu.goorm.io/lecture/25552/파이썬-개발자를-위한...,프로그래밍 - 프로그래밍 기초
7,[하루 10분|Web Project] HTML/JS/CSS로 나만의 MBTI 사이트...,무료,판다코딩,https://edu.goorm.io/lecture/25652/하루-10분-web-...,웹 프로그래밍 - 프로그래밍
8,React & TypeScript & GraphQL 로 버그없이 빠르게 개발하기!,"₩25,000",Alucard,https://edu.goorm.io/lecture/25571/react-types...,웹 프로그래밍 - 프로그래밍
9,파이썬으로 구현하는 CF 알고리즘,"₩35,100",최욱진,https://edu.goorm.io/lecture/25544/파이썬으로-구현하는-...,프로그래밍 - 인공지능


In [151]:
df.to_csv("구름edu_2페이지.csv", encoding="utf-8-sig")

### 3페이지

In [153]:
title_list3 = []
price_list3 = []
teacher_list3 = []
rating_list3 = []
url_list3 = []

for i in tqdm(range(3,4)):
    url1 = "https://edu.goorm.io/category/programming?page="+str(i)+"&sort=newest"
    driver = wb.Chrome()
    driver.get(url1)
    soup = bs(driver.page_source,"lxml")
    time.sleep(1)
    price = soup.select("div.RfUd-z._1AV9t8.card-footer > div > span._3vh60A")
    title = soup.select("div > div.obzTi8 > div.kV2LiJ")
    teacher = soup.select('._3SzRPA > .xaJHLa')
    rating = soup.select('._2KWt9f._3SwFuE')
    url = soup.select("._1xnzzp")
    time.sleep(1)
    
    for i in range(len(title)):
        title_list3.append(title[i].text)
        price_list3.append(price[i].text)
        teacher_list3.append(teacher[i].text)
        rating_list3.append(rating[i].text)
        url_list3.append('https://edu.goorm.io'+url[i]['href'])
    time.sleep(1)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:10<00:00, 10.96s/it]


In [198]:
cate_list3 = []
for k in tqdm(range(3,4)):
    for i in tqdm(range(1,21)):
        url1 = "https://edu.goorm.io/category/programming?page="+str(k)+"&sort=newest"
        driver = wb.Chrome()
        driver.get(url1)
        soup = bs(driver.page_source,"lxml")
        time.sleep(1)
        btn = driver.find_element_by_css_selector('div.Aj2j_L > div:nth-child('+str(i)+')>a')
        btn.click()
        time.sleep(3)
        soup = bs(driver.page_source,"lxml")
        #카테고리(cate)
        cate = soup.select('div:nth-child(2) > div._2yM5um')
        #카테고리 저장
        for j in range(len(cate)):
            cate_list3.append(cate[j].text)
        driver.close()

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [03:44<00:00, 224.44s/it]


In [199]:
dic3 = {'제목':title_list3, '가격':price_list3, '강사':teacher_list3, 'URL': url_list3, '카테고리':cate_list3}
df = pd.DataFrame(dic3)
df

,제목,가격,강사,URL,카테고리
0,2021 온라인 코딩 파티 시즌1 - 파이썬 데이터 분석 및 시각화!!!,무료,구름,https://edu.goorm.io/lecture/24790/2021-온라인-코딩...,프로그래밍 - 데이터 추출/분석
1,"2021 온라인 코딩 파티 시즌1 - 둘이서 코딩, 거북선과 척후선!!!",무료,구름,https://edu.goorm.io/lecture/24791/2021-온라인-코딩...,프로그래밍 - 블록 프로그래밍
2,2021 온라인 코딩 파티 시즌1 - 코딩랜드를 구하라!!!,무료,구름,https://edu.goorm.io/lecture/24792/2021-온라인-코딩...,프로그래밍 - 블록 프로그래밍
3,2021 온라인 코딩 파티 시즌1 - 도전 틱택토!!!,무료,구름,https://edu.goorm.io/lecture/24793/2021-온라인-코딩...,프로그래밍 - 프로그래밍 기초
4,[애플 공식 교재] 스위프트 퍼즐 2 - AD free,"₩11,000",코더스하이,https://edu.goorm.io/lecture/24597/애플-공식-교재-스위...,프로그래밍 - 프로그래밍 기초
5,[애플 공식 교재] 스위프트 퍼즐 1 - AD free,"₩11,000",코더스하이,https://edu.goorm.io/lecture/24579/애플-공식-교재-스위...,프로그래밍 - 프로그래밍 기초
6,스프링부트 개념정리,무료,데어 프로그래밍,https://edu.goorm.io/lecture/24604/스프링부트-개념정리,웹 프로그래밍 - 프로그래밍
7,스프링부트 나만의 블로그 만들기,무료,데어 프로그래밍,https://edu.goorm.io/lecture/24605/스프링부트-나만의-블...,웹 프로그래밍 - 프로그래밍
8,스프링부트 시큐리티 특강,무료,데어 프로그래밍,https://edu.goorm.io/lecture/24606/스프링부트-시큐리티-특강,웹 프로그래밍 - 프로그래밍
9,[따라하며 배우는 리액트&파이어베이스] 채팅 애플리케이션 만들기,"₩31,900",John Ahn,https://edu.goorm.io/lecture/24672/따라하며-배우는-리액...,웹 프로그래밍 - 프로그래밍


In [155]:
df.to_csv("구름edu_3페이지.csv", encoding="utf-8-sig")

### 4페이지부터 끝까지

In [159]:
title_list4 = []
price_list4 = []
teacher_list4 = []
rating_list4 = []
url_list4 = []

for i in tqdm(range(4,21)):
    url1 = "https://edu.goorm.io/category/programming?page="+str(i)+"&sort=newest"
    driver = wb.Chrome()
    driver.get(url1)
    soup = bs(driver.page_source,"lxml")
    time.sleep(1)
    price = soup.select("div.RfUd-z._1AV9t8.card-footer > div > span._3vh60A")
    title = soup.select("div > div.obzTi8 > div.kV2LiJ")
    teacher = soup.select('._3SzRPA > .xaJHLa')
    rating = soup.select('._2KWt9f._3SwFuE')
    url = soup.select("._1xnzzp")
    time.sleep(1)
    
    for i in range(len(title)):
        title_list4.append(title[i].text)
        price_list4.append(price[i].text)
        teacher_list4.append(teacher[i].text)
        rating_list4.append(rating[i].text)
        url_list4.append('https://edu.goorm.io'+url[i]['href'])
    time.sleep(1)
    driver.close()

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [03:38<00:00, 12.86s/it]


In [156]:
cate_list4 = []
for k in tqdm(range(4,21)):
    for i in tqdm(range(1,21)):
        url1 = "https://edu.goorm.io/category/programming?page="+str(k)+"&sort=newest"
        driver = wb.Chrome()
        driver.get(url1)
        soup = bs(driver.page_source,"lxml")
        time.sleep(1)
        btn = driver.find_element_by_css_selector('div.Aj2j_L > div:nth-child('+str(i)+')>a')
        btn.click()
        time.sleep(3)
        soup = bs(driver.page_source,"lxml")
        #카테고리(cate)
        cate = soup.select('div:nth-child(2) > div._2yM5um')
        #카테고리 저장
        for j in range(len(cate)):
            cate_list4.append(cate[j].text)
        driver.close()

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [03:30<00:00, 10.52s/it]


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [03:49<00:00, 11.49s/it]


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [03:45<00:00, 11.30s/it]


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [03:21<00:00, 10.08s/it]


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [03:58<00:00, 11.93s/it]


 94%|██████████████████████████████████████████████████████████████████████████▎    | 16/17 [1:01:06<03:49, 229.18s/it]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.Aj2j_L > div:nth-child(13)>a"}
  (Session info: chrome=94.0.4606.81)


In [226]:
print(len(title_list4))
print(len(price_list4))
print(len(teacher_list4))
print(len(rating_list4))
print(len(url_list4))
print(len(cate_list4))

332
332
332
332
332
332


In [173]:
dic3 = {'제목':title_list4, '가격':price_list4, '강사':teacher_list4, 'URL': url_list4, '카테고리':cate_list4}
df = pd.DataFrame(dic3)
df

,제목,가격,강사,URL,카테고리
0,[애플 공식 교재] iOS 앱 만들기 입문,"₩38,500",코더스하이,https://edu.goorm.io/lecture/23901/애플-공식-교재-io...,프로그래밍 - 모바일 프로그래밍
1,파이썬으로 하는 데이터 분석 - Pandas 제대로 배우기,"₩36,000",아이티고,https://edu.goorm.io/lecture/23732/파이썬으로-하는-데이...,프로그래밍 - 데이터 추출/분석
2,[2020 NEW] 코틀린 기반 안드로이드 앱 개발 Part3 - 데이터 관리와 네트워크,"₩66,000",소프트캠퍼스,https://edu.goorm.io/lecture/23730/2020-new-코틀...,프로그래밍 - 모바일 프로그래밍
3,[2020 NEW] 코틀린 기반 안드로이드 앱 개발 Part2 - 메뉴와 4대 구성요소,"₩66,000",소프트캠퍼스,https://edu.goorm.io/lecture/23729/2020-new-코틀...,프로그래밍 - 모바일 프로그래밍
4,파이썬 기반 판다스(Pandas)를 활용한 데이터 분석,"₩66,000",소프트캠퍼스,https://edu.goorm.io/lecture/23731/파이썬-기반-판다스-...,프로그래밍 - 데이터 추출/분석
...,...,...,...,...,...
327,제1회 구름코드챌린지(GCC) 기출 문제,무료,구름,https://edu.goorm.io/lecture/790/제1회-구름코드챌린지-g...,프로그래밍 - 프로그래밍 교양
328,[바로실습] 생활코딩 - 파이썬(Python),무료,생활코딩,https://edu.goorm.io/lecture/44/바로실습-생활코딩-파이썬-...,프로그래밍 - 프로그래밍 기초
329,한 눈에 끝내는 아두이노 기초,무료,구름,https://edu.goorm.io/lecture/203/한-눈에-끝내는-아두이노-기초,프로그래밍 - IoT
330,10주 완성 알고리즘 코딩테스트,"₩200,000",코딩몬스터,https://edu.goorm.io/lecture/554/10주-완성-알고리즘-코...,프로그래밍 - 알고리즘


In [174]:
df.to_csv("구름edu_4-20페이지.csv", encoding="utf-8-sig")

### 리뷰 시도

### 연습

# 2페이지

In [216]:
goorum_review_list2= []

for i in tqdm(range(2,3)):
    for k in tqdm(range(1,21)):
        #창열기
        url1 = "https://edu.goorm.io/category/programming?page="+str(i)+"&sort=newest"
        driver = wb.Chrome()
        driver.get(url1)
        soup = bs(driver.page_source,"lxml")
        #클릭할 항목 수 변동
        btn = driver.find_element_by_css_selector('div.Aj2j_L > div:nth-child('+str(k)+')>a')
        btn.click()
        time.sleep(3)
        soup = bs(driver.page_source,"lxml")
        time.sleep(1)
        #리뷰를 한꺼번에 모아줄 리스트 초기화
        review_list = []
        #리뷰 3개 넣기
        for j in range(2,5):
            review = soup.select('#tabContents_4 > div > ul > li:nth-child('+str(j)+') > div > div._21ON4p.card-body > p')
        #리뷰 3개를 한꺼번에 모아줄 review_list에 넣기
        for a in range(len(review)):
            review_list.append(review[a].text)
        goorum_review_list2.append(review_list)
        driver.close()

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [04:39<00:00, 279.15s/it]


In [217]:
dic2 = {'제목':title_list2, '가격':price_list2, '강사':teacher_list2, 'URL': url_list2, '카테고리':cate_list2, '리뷰':goorum_review_list2}
df = pd.DataFrame(dic2)
df

,제목,가격,강사,URL,카테고리,리뷰
0,실전! 웹사이트 제작 Step by Step - 호텔 라한 Application C...,"₩27,500",J.young,https://edu.goorm.io/lecture/26324/실전-웹사이트-제작-...,웹 프로그래밍 - 프로그래밍,[]
1,딥러닝 네트워크의 연산,"₩39,000",Shin's Lab,https://edu.goorm.io/lecture/26039/딥러닝-네트워크의-연산,프로그래밍 - 인공지능,[]
2,데이터 시각화를 위한 파이썬,"₩39,000",Shin's Lab,https://edu.goorm.io/lecture/24603/데이터-시각화를-위한...,프로그래밍 - 데이터 추출/분석,[]
3,딥러닝을 위한 파이썬 레벨2,"₩39,000",Shin's Lab,https://edu.goorm.io/lecture/26038/딥러닝을-위한-파이썬...,프로그래밍 - 인공지능,[]
4,딥러닝을 위한 파이썬 레벨1,"₩35,100",Shin's Lab,https://edu.goorm.io/lecture/26037/딥러닝을-위한-파이썬...,프로그래밍 - 인공지능,[]
5,모의해킹 취업 준비하는데 알아두면 좋은 것,무료,보안프로젝트,https://edu.goorm.io/lecture/25879/모의해킹-취업-준비하...,프로그래밍 - 보안,[]
6,파이썬 개발자를 위한 go언어 기초,무료,행보칸코딩,https://edu.goorm.io/lecture/25552/파이썬-개발자를-위한...,프로그래밍 - 프로그래밍 기초,[go 언어는 처음 접해보는데 익숙한 파이썬과 비교해주시니 더욱 쉽게 이해할 수 있...
7,[하루 10분|Web Project] HTML/JS/CSS로 나만의 MBTI 사이트...,무료,판다코딩,https://edu.goorm.io/lecture/25652/하루-10분-web-...,웹 프로그래밍 - 프로그래밍,[]
8,React & TypeScript & GraphQL 로 버그없이 빠르게 개발하기!,"₩25,000",Alucard,https://edu.goorm.io/lecture/25571/react-types...,웹 프로그래밍 - 프로그래밍,[]
9,파이썬으로 구현하는 CF 알고리즘,"₩35,100",최욱진,https://edu.goorm.io/lecture/25544/파이썬으로-구현하는-...,프로그래밍 - 인공지능,[]


In [218]:
df.to_csv("구름edu_2페이지.csv", encoding="utf-8-sig")

### 3페이지

In [223]:
#리스트 초기화
goorum_review_list3 = []

for i in tqdm(range(3,4)):
    for k in tqdm(range(1,21)):
        #창열기
        url1 = "https://edu.goorm.io/category/programming?page="+str(i)+"&sort=newest"
        driver = wb.Chrome()
        driver.get(url1)
        soup = bs(driver.page_source,"lxml")
        #클릭할 항목 수 변동
        btn = driver.find_element_by_css_selector('div.Aj2j_L > div:nth-child('+str(k)+')>a')
        btn.click()
        time.sleep(3)
        soup = bs(driver.page_source,"lxml")
        time.sleep(1)
        #리뷰를 한꺼번에 모아줄 리스트 초기화
        review_list = []
        #리뷰 3개 넣기
        for j in range(2,5):
            review = soup.select(' #tabContents_3> div > ul > li:nth-child('+str(j)+') > div > div._21ON4p.card-body > p')
        #리뷰 3개를 한꺼번에 모아줄 review_list에 넣기
        for a in range(len(review)):
            review_list.append(review[a].text)
        goorum_review_list3.append(review_list)
        driver.close()

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [04:26<00:00, 266.99s/it]


In [224]:
dic3 = {'제목':title_list3, '가격':price_list3, '강사':teacher_list3, 'URL': url_list3, '카테고리':cate_list3, '리뷰': goorum_review_list3}
df = pd.DataFrame(dic3)
df

,제목,가격,강사,URL,카테고리,리뷰
0,2021 온라인 코딩 파티 시즌1 - 파이썬 데이터 분석 및 시각화!!!,무료,구름,https://edu.goorm.io/lecture/24790/2021-온라인-코딩...,프로그래밍 - 데이터 추출/분석,[:))))]
1,"2021 온라인 코딩 파티 시즌1 - 둘이서 코딩, 거북선과 척후선!!!",무료,구름,https://edu.goorm.io/lecture/24791/2021-온라인-코딩...,프로그래밍 - 블록 프로그래밍,[와우 너무 재밌어요\n]
2,2021 온라인 코딩 파티 시즌1 - 코딩랜드를 구하라!!!,무료,구름,https://edu.goorm.io/lecture/24792/2021-온라인-코딩...,프로그래밍 - 블록 프로그래밍,[똥싸세요]
3,2021 온라인 코딩 파티 시즌1 - 도전 틱택토!!!,무료,구름,https://edu.goorm.io/lecture/24793/2021-온라인-코딩...,프로그래밍 - 프로그래밍 기초,[ㅋㅋ 심심풀이로 들어봤는데 재밌네요.]
4,[애플 공식 교재] 스위프트 퍼즐 2 - AD free,"₩11,000",코더스하이,https://edu.goorm.io/lecture/24597/애플-공식-교재-스위...,프로그래밍 - 프로그래밍 기초,[]
5,[애플 공식 교재] 스위프트 퍼즐 1 - AD free,"₩11,000",코더스하이,https://edu.goorm.io/lecture/24579/애플-공식-교재-스위...,프로그래밍 - 프로그래밍 기초,[]
6,스프링부트 개념정리,무료,데어 프로그래밍,https://edu.goorm.io/lecture/24604/스프링부트-개념정리,웹 프로그래밍 - 프로그래밍,[개념잡기에 너무 좋습니다]
7,스프링부트 나만의 블로그 만들기,무료,데어 프로그래밍,https://edu.goorm.io/lecture/24605/스프링부트-나만의-블...,웹 프로그래밍 - 프로그래밍,[설명이 너무 잘되어있어요\n]
8,스프링부트 시큐리티 특강,무료,데어 프로그래밍,https://edu.goorm.io/lecture/24606/스프링부트-시큐리티-특강,웹 프로그래밍 - 프로그래밍,[]
9,[따라하며 배우는 리액트&파이어베이스] 채팅 애플리케이션 만들기,"₩31,900",John Ahn,https://edu.goorm.io/lecture/24672/따라하며-배우는-리액...,웹 프로그래밍 - 프로그래밍,[]


In [225]:
df.to_csv("구름edu_3페이지.csv", encoding="utf-8-sig")

### 4-20페이지

In [228]:
#리스트 초기화
goorum_review_list4 = []

for i in tqdm(range(4,21)):
    for k in tqdm(range(1,21)):
        #창열기
        url1 = "https://edu.goorm.io/category/programming?page="+str(i)+"&sort=newest"
        driver = wb.Chrome()
        driver.get(url1)
        soup = bs(driver.page_source,"lxml")
        #클릭할 항목 수 변동
        btn = driver.find_element_by_css_selector('div.Aj2j_L > div:nth-child('+str(k)+')>a')
        btn.click()
        time.sleep(3)
        soup = bs(driver.page_source,"lxml")
        time.sleep(1)
        #리뷰를 한꺼번에 모아줄 리스트 초기화
        review_list = []
        #리뷰 3개 넣기
        for j in range(2,5):
            review = soup.select(' #tabContents_3> div > ul > li:nth-child('+str(j)+') > div > div._21ON4p.card-body > p')
        #리뷰 3개를 한꺼번에 모아줄 review_list에 넣기
        for a in range(len(review)):
            review_list.append(review[a].text)
        goorum_review_list4.append(review_list)
        driver.close()

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [04:33<00:00, 13.66s/it]


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [04:23<00:00, 13.19s/it]


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [04:30<00:00, 13.52s/it]


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [04:28<00:00, 13.41s/it]


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [04:32<00:00, 13.60s/it]


 94%|██████████████████████████████████████████████████████████████████████████▎    | 16/17 [1:14:21<04:38, 278.86s/it]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.Aj2j_L > div:nth-child(13)>a"}
  (Session info: chrome=94.0.4606.81)


In [229]:
dic4 = {'제목':title_list4, '가격':price_list4, '강사':teacher_list4, 'URL': url_list4, '카테고리':cate_list4, '리뷰':goorum_review_list4}
df = pd.DataFrame(dic4)
df

,제목,가격,강사,URL,카테고리,리뷰
0,[애플 공식 교재] iOS 앱 만들기 입문,"₩38,500",코더스하이,https://edu.goorm.io/lecture/23901/애플-공식-교재-io...,프로그래밍 - 모바일 프로그래밍,[]
1,파이썬으로 하는 데이터 분석 - Pandas 제대로 배우기,"₩36,000",아이티고,https://edu.goorm.io/lecture/23732/파이썬으로-하는-데이...,프로그래밍 - 데이터 추출/분석,[]
2,[2020 NEW] 코틀린 기반 안드로이드 앱 개발 Part3 - 데이터 관리와 네트워크,"₩66,000",소프트캠퍼스,https://edu.goorm.io/lecture/23730/2020-new-코틀...,프로그래밍 - 모바일 프로그래밍,[]
3,[2020 NEW] 코틀린 기반 안드로이드 앱 개발 Part2 - 메뉴와 4대 구성요소,"₩66,000",소프트캠퍼스,https://edu.goorm.io/lecture/23729/2020-new-코틀...,프로그래밍 - 모바일 프로그래밍,[]
4,파이썬 기반 판다스(Pandas)를 활용한 데이터 분석,"₩66,000",소프트캠퍼스,https://edu.goorm.io/lecture/23731/파이썬-기반-판다스-...,프로그래밍 - 데이터 추출/분석,[]
...,...,...,...,...,...,...
327,제1회 구름코드챌린지(GCC) 기출 문제,무료,구름,https://edu.goorm.io/lecture/790/제1회-구름코드챌린지-g...,프로그래밍 - 프로그래밍 교양,[]
328,[바로실습] 생활코딩 - 파이썬(Python),무료,생활코딩,https://edu.goorm.io/lecture/44/바로실습-생활코딩-파이썬-...,프로그래밍 - 프로그래밍 기초,[]
329,한 눈에 끝내는 아두이노 기초,무료,구름,https://edu.goorm.io/lecture/203/한-눈에-끝내는-아두이노-기초,프로그래밍 - IoT,[]
330,10주 완성 알고리즘 코딩테스트,"₩200,000",코딩몬스터,https://edu.goorm.io/lecture/554/10주-완성-알고리즘-코...,프로그래밍 - 알고리즘,[]


In [230]:
df.to_csv("구름edu_4-20페이지.csv", encoding="utf-8-sig")